<font size=25>Laboratory 3 summary</font>

In this lab, we will:

I: Analyse different optimizers
  * Build intuition on how the most popular optimizers work
  * Implement classic Stochastic Gradient Descent (`SGD`)
  * Add momentum to the classic `SGD` to obtain `MomentumSGD`. How does momentum help?
  * Compare `SGD`, `MomentumSGD` with `Adam`, a widely used optimizer in practice

II: Train a Neural Network and familiarise ourselves with PyTorch:
  * learn to create a `Dataset` and `Dataloader` for datasets
  * write train and validation loops over batches
  * plot learning and accuracy curves
  * inspect confusion matrix
  * inspect gradient norms
  * deal with unbalanced datasets during training

# Part I: Optimization dynamics

We are interested in finding parameter values for our models that minimize the loss function L, for a dataset D.

$$\hat{\theta} = \arg\max_{\theta}[L(\theta, D)] $$

Recall the gradient of the loss function with respects to parameters $\theta$ for an input example x:

$$
∇_\theta L(\theta, x^{(i)}) = [\frac{∂L}{\partial{\theta_1}}, ... ,\frac{∂L}{\partial{\theta_n}}]^T
$$

For visualization purposes, in this section we will use a model with two weights and analyse the trajectory given by our optimizers on the 2D surface of the function it models.

## Some theory

<!-- $$\theta^{(t+1)} = \theta^{(t)} - α∇_\theta\underbrace{\left(\frac{1}{N}\sum_{i=1}^{N}  L(\theta^{(t)}, x^{(i)})\right)}_\text{loss over all dataset examples}$$ -->

We will take a look at three optimization algorithms: the classic `SGD`, `SGD` with momentum and `Adam`.

### SGD
$$g^{(t+1)} = \underbrace{∇_\theta L(\theta^{(t)}, x^{(i)})}_\text{current gradient}$$
$$\theta^{(t+1)} = \theta^{(t)} - \underbrace{α}_\text{learning rate}g^{(t+1)}$$

* "Stochastic" because $x^{(i)}$ are not the entire dataset, but a randomly sampled batch (more on this in part II)
* Often used with a learning rate scheduler in practice (α is not constant in time)

### SGD with momentum

* Standard `SGD` might take a very indirect path towards the minimum

* We introduce momentum: a combination of the gradient of the current step and the direction moved in the previous step, weighted by a factor β

$$ m^{(t+1)} = \overbrace{\beta}^\text{momentum factor} \underbrace{m^{(t)}}_\text{momentum} + (1 - {\beta}) \underbrace{∇_\theta L(\theta^{(t)}, x^{(i)})}_\text{current gradient}$$

$$\theta^{(t+1)} = \theta^{(t)} - α m^{(t+1)}$$

* The overall effect of the momentum term is a smoother trajectory and reduced oscillatory behavior in valley-like regions

|![SGDmomentum](https://production-media.paperswithcode.com/methods/Screen_Shot_2020-05-28_at_3.25.40_PM_Y687HvA.png)|
|:--:|
| SGD trajectories with and without momentum ([source](https://www.researchgate.net/publication/333469047_The_Frontier_of_SGD_and_Its_Variants_in_Machine_Learning)) |

### Nesterov accelerated momentum
* Computes gradient at the prediction point instead of the current point (performs a lookahead step)
* One way to think about this is that the gradient term now corrects the path provided by momentum alone

$$ m^{(t+1)} = \overbrace{\beta}^\text{momentum factor} \underbrace{m^{(t)}}_\text{momentum} + (1 - {\beta}) \underbrace{∇_\theta L(\theta^{(t)} - \alpha m^{(t)}, x^{(i)})}_\text{gradient at the prediction step}$$

$$\theta^{(t+1)} = \theta^{(t)} - α m^{(t+1)}$$


|![Nesterov Momentum](https://cs231n.github.io/assets/nn3/nesterov.jpeg)|
|:--:|
| Standard vs Nesterov Momentum (credit: Stanford CS231) ([source](https://cs231n.github.io/assets/nn3/nesterov.jpeg)) |



Just some imports we will need throughout the lab.

In [ ]:
# Some snippets taken from Google tutorial:
# https://colab.research.google.com/github/google/eng-edu/blob/master/ml/pc/exercises/image_classification_part1.ipynb#scrollTo=H4XHh2xSfgie
from __future__ import print_function, division
import os
import torch
import random
from torchvision.transforms import ToTensor, ToPILImage
import zipfile
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import RandomSampler, Sampler
from torchvision import transforms, utils
import torch.nn as nn
from tqdm import tqdm
from typing import Iterator, List, Callable, Tuple
from functools import partial
from math import *
from IPython.display import HTML

from matplotlib import rc, cm
rc('animation', html='jshtml')

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.animation as animation
%matplotlib notebook

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

First, something that might come in handy (especially for assignment): ProfileReport

In [ ]:
!pip install -q ydata-profiling

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.5/393.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.8/687.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 26.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport

# Create a toy dataset
data = {
    "Name": ["Alice", "Bob", "Charlie", "David", "Eve"],
    "Age": [25, 30, 35, 40, 28],
    "Salary": [50000, 60000, 75000, 80000, 62000],
    "City": ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix"]
}

df = pd.DataFrame(data)

# Generate a profile report
profile = ProfileReport(df, explorative=True)

# Save report to an HTML file
profile.to_file("profile_report.html")

# To display in a Jupyter Notebook
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## SGD (ToDo)

Implement the `step()` function of classic `SGD` using the above formula for a simple model with two parameters $w_1$ and $w_2$.

In [ ]:
class SGD:
    """
    Stochastic Gradient Descent optimizer (2 parameters)
    """
    def __init__(self, params: Iterator[nn.Parameter], lr: float=0.01):
        self.w1, self.w2 = list(params)
        self.lr = lr

    def step(self):
        """
        Perform a gradient decent step. Update the parameters w1 and w2 by using:
          - the gradient of the loss with respect to the parameters
          - the learning rate
        This method is called after backward(), so the gradient of the loss wrt
        the parameters is already computed and stored.
        """
        with torch.no_grad():
            # TODO: update the weights w1 and w2
            # Hint: when we call .backward(), our gradients are stored in weight.grad

    def zero_grad(self):
        self.w1.grad.zero_()
        self.w2.grad.zero_()

## SGD with momentum (ToDo)
Add momentum: Implement the `step()` method of `SGDMomentum` using the above formula.

In [ ]:
class SGDMomentum:
    """
    Stochastic Gradient Descent optimizer (2 parameters) with momentum
    """
    def __init__(self,
                params: Iterator[nn.Parameter],
                β: float=0.9,
                lr: float=0.01):
      self.w1, self.w2 = list(params)
      self.lr = lr
      self.β = β
      self.m_w1 = None # previous gradient for w1
      self.m_w2 = None # previous gradient for w2

    def step(self):
      """
      TODO: Perform a gradient decent step with momentum. Update the parameters
      w1 and w2 by using:
      - the gradient of the loss with respect to the parameters
      - the learning rate
      - the momentum factor β
      This method is called after backward(), so the gradient of the loss wrt
      the parameters is already computed and stored.
      """
      with torch.no_grad():
          # we have computed the gradient before
          if self.m_w1:
              # TODO update gradient for w1 and w2 using the momentum equation
              # TODO update parameters w1 and w2
          else:
              # it's the first time we compute the gradient, so we save it for
              # future momentum updates
              # we initialize the momentum with the current gradients
              # update parameters for w1 and w2

    def zero_grad(self):
        self.w1.grad.zero_()
        self.w2.grad.zero_()

## More advanced optimizers: RMSProp and Adam

One problem with `SGD` is that the adjustment is proportional with the graident magnitude for each parameter. More advanced optimizers use adaptive learning rates to mitigate the problem.

* `RMSProp`: maintaining a moving average of the squares of gradients and adjust the learning rate by this average:

$$ m^{(t+1)} = \underbrace{∇_\theta L(\theta^{(t)}, x^{(i)})}_\text{current gradient}$$

$$ v^{(t+1)} = γ v^{(t)} + (1-γ) (\underbrace{∇_\theta L(\theta^{(t)}, x^{(i)})}_\text{pointwise square gradient})^2$$


$$\theta^{(t+1)} = \theta^{(t)} - \dfrac{α}{\sqrt{v^{(t+1)}} + ϵ} m^{(t+1)}$$


Here, $\epsilon$ is a small constant that prevents zero division. By applying the division with the square root of the pointwise square gradient, the result is that each component is normalised and we adjust each parameter by a fixed step ($\alpha$), with the sign of the gradient deciding the direction.

* `Adam` (Adaptive moment estimation) combines the adaptive learning rate with momentum, by adding momentum to both the estimates of the gradient and the squared gradient, with coefficients $β_1$ and $β_2$ in [0,1):

$$ m^{(t+1)} = β_1 m^{(t)} + (1 - β_1) ∇_\theta L(\theta^{(t)}, x^{(i)})$$

$$ v^{(t+1)} = β_2 v^{(t)} + (1 - β_2) (∇_\theta L(\theta^{(t)}, x^{(i)}))^2$$

(Q) Is there a problem with the current implementation?

Yes! At the first iteration, all the measurements are zero, therefore we add bias to our estimates towards small values. We need to bias-correct the terms:

$$ \tilde{m}^{(t+1)} = \dfrac{m^{(t+1)}}{1 - β_1^{t+1}} $$

$$ \tilde{v}^{(t+1)} = \dfrac{v^{(t+1)}}{1 - β_2^{t+1}} $$

We therefore update the parameters as:

$$\theta^{(t+1)} = \theta^{(t)} - \dfrac{α}{\sqrt{\tilde{v}^{(t+1)}} + ϵ} \tilde{m}^{(t+1)}$$


In [ ]:
class Adam:
    """
    Adaptive moment estimation
    """
    def __init__(self, params: Iterator[nn.Parameter],
                 lr: float=0.05,
                 β1=0.9,
                 β2=0.999):
        self.w1, self.w2 = list(params)
        self.lr = lr
        self.β1 = β1
        self.β2 = β2
        self.ϵ = 1e-8
        self.mt_w1 = 0
        self.vt_w1 = 0
        self.mt_w2 = 0
        self.vt_w2 = 0
        self.t = 1

    def step(self):
        with torch.no_grad():
            # TODO compute the update step using the above formulas for Adam
            self.mt_w1 = ...
            self.mt_w2 = ...

            self.vt_w1 = ...
            self.vt_w2 = ...

            mt_hat_w1 = self.mt_w1 / (1 - self.β1**self.t)
            mt_hat_w2 = self.mt_w2 / (1 - self.β1**self.t)

            vt_hat_w1 = self.vt_w1 / (1 - self.β2**self.t)
            vt_hat_w2 = self.vt_w2 / (1 - self.β2**self.t)

            self.w1 -= ...
            self.w2 -= ...

            self.t += 1

    def zero_grad(self):
        self.w1.grad.zero_()
        self.w2.grad.zero_()

## Some didactic functions
We define several classical two-parameter functions to simulate the loss surface for our two-parameter model.

We study the evolution of the parameters using the above optimizers and inspecting the trajectories from an initial point, performing update steps.

TODO: Try out different functions, change the starting point and learning rate and visualize the optimizers trajectories.

In [ ]:
def mexican(x, y):
    if type(x) != torch.nn.parameter.Parameter:
        return np.sin(np.sqrt(x ** 2 + y ** 2))
    else:
        return torch.sin(torch.sqrt(x ** 2 + y ** 2))

def three_hump_camel(x, y):
    return 2*(x**2) - 1.05*(x**4) + (x**6)/6 + x*y + y**2

def rosenbrock(x, y):
    a, b = 1, 20
    return (a-x)**2 + b*(y-x**2)**2

def saddle(x, y):
    return x*x - y*y

def hills(x, y):
    if type(x) != torch.nn.parameter.Parameter:
        return -1 * np.sin(x * x) * np.cos(3 * y * y) * np.exp(
            -(x * y) * (x * y)
        ) - np.exp(-(x + y) * (x + y))
    else:
        return -1 * torch.sin(x * x) * torch.cos(3 * y * y) * torch.exp(
            -(x * y) * (x * y)
        ) - torch.exp(-(x + y) * (x + y))

In [ ]:
"""
For each function we define:
  - a plotting [x,y] area (interval)
  - a minima (fmin)
  - a starting point (fstart)
"""
functions = {
    "saddle": {
        "f": saddle,
        "interval": ((-4.5, 4.5), (-4.5, 4.5)),
        "fmin": (0.0, -4.5),
        "fstart": (-3.7, -1.5)
    },
    "mexican": {
        "f": mexican,
        "interval": ((-2.0, 2.0), (-1.0, 3.0)),
        "fmin": (0.0, 0.0),
        "fstart": (-0.9, 0.7)
    },
    "three_hump_camel": {
        "f": three_hump_camel,
        "interval": ((-2.0, 2.0), (-2.0, 2.0)),
        "fmin": (0.0, 0.0),
        "fstart": (1.0, 1.9)
    },
    "rosenbrock": {
        "f": rosenbrock,
        "interval": ((-2.0, 2.0), (-1.0, 3.0)),
        "fmin": (1.0, 1.0),
        "fstart": (0.0, 1.5)
    },
    "hills": {
        "f": hills,
        "interval": ((-2.0, 2.0), (-1.0, 3.0)),
        "fmin": (1.0, 1.0),
        "fstart": (-1.0, 2.5)
    }
}

## Loss landscape visualization
Now, let's check our optimizers in action with some nice animations :)

In [ ]:
# Here we up some parameters for each optimizer. We will instantiate obj and pos later
optim_data = [
    {
        'name': 'SGD',
        'class': SGD,
        'args': dict(lr=0.01),
        'plot_color': 'g',
        'obj': None,
        'pos': None
    },
    {
        'name': 'SGDMomentum',
        'class': SGDMomentum,
        'args': dict(lr=0.01, β=0.85),
        'plot_color': 'm',
        'obj': None,
        'pos': None
    },
    {
        'name': 'Adam',
        'class': Adam,
        'args': dict(lr=0.05, β1=0.9, β2=0.999),
        'plot_color': 'r',
        'obj': None,
        'pos': None
    }
]

In [ ]:
# Crate the function surface, using the predefined intervals and set up the starting points.
f_name = "three_hump_camel"
f = functions[f_name]["f"]
interval = functions[f_name]["interval"]
f_min = functions[f_name]["fmin"]
f_start = functions[f_name]["fstart"]

# setup starting point
_x0, _y0 = f_start
_z0 = f(_x0, _y0)

# compute function values on the predefined interval
x = np.linspace(*interval[0], 30)
y = np.linspace(*interval[1], 30)

x_mesh, y_mesh = np.meshgrid(x, y)
z_mesh = f(x_mesh, y_mesh)

In [ ]:
# We will call this function to generate frames in the animation
def frame(w):
    global optim_data

    for oix, optim_dict in enumerate(optim_data):
        optim = optim_dict['obj']
        [x0, y0, z0] = optim_dict['pos']

        w1 = optim.w1
        w2 = optim.w2
        _z = f(w1, w2)
        _z = _z.squeeze()
        _z.backward()
        _z = _z.item()

        # parameters after update
        optim.step()
        optim.zero_grad()

        x = optim.w1.item()
        y = optim.w2.item()
        z = f(x, y)

        # plot the trajectory
        plot = ax.plot(
            [x0, x],
            [y0, y],
            [z0, z],
            linewidth=2, color=optim_dict['plot_color'], alpha=1, zorder=10
        )

        # update and plot the tip of the trajectoy
        tips[oix].remove()
        tips[oix] = ax.scatter(
            x, y, z,
            s=20, label=optim_dict['name'], color=optim_dict['plot_color'],
            alpha=1, zorder=10, depthshade=False
        )

        # update current position in each optimzier state
        optim_dict['pos'] = [x, y, z]

    return plot

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection='3d')

ax.plot_surface(x_mesh, y_mesh, z_mesh,
                alpha=.8, rstride=1, cstride=1,
                cmap=cm.coolwarm, edgecolor='none')
ax.scatter(*f_min, f(*f_min), s=100, alpha=1.0, label='global min', color='r')
ax.scatter(*f_start, f(*f_start), s=100, alpha=1.0, label='start', color='g')

tips = [None] * len(optim_data)

# Prepare the optimizers
for oix, optim_params in enumerate(optim_data):
    w1 = nn.Parameter(torch.tensor([_x0]))
    w2 = nn.Parameter(torch.tensor([_y0]))
    optimizer = optim_params['class'](params=[w1, w2], **optim_params['args'])

    # Here we instantiate the optimizers and the initial positions
    optim_params['obj'] = optimizer
    optim_params['pos'] = [_x0, _y0, _z0]

    tips[oix] = ax.scatter(
        _x0, _y0, _z0, s=20,
        depthshade=True, label=optim_params['name'], color=optim_params['plot_color']
    )

ax.legend(tips, [o['name'] for o in optim_data])
ax.view_init(elev=55, azim=-78)

anim = animation.FuncAnimation(fig, frame, frames=50, blit=False, repeat=True)
HTML(anim.to_html5_video())

# Part II: PyTorch datasets and dataloaders

## Gradient descent vs mini-batch gradient descent

### Gradient descent
To optimize the neural network parameters $\theta$, we can use the gradient descent method, in which we compute the gradient of the loss $L$ with respect to the parameters $\theta$, then update the parameters $\theta$ in the opposite direction of the gradient with a learning rate $\alpha$.

To compute the gradient we use the whole dataset $x^{(1)}$, $x^{(2)}$, ..., $x^{(N)}$:
$$\theta^{(t+1)} = \theta^{(t)} - α∇_\theta\underbrace{\left(\frac{1}{N}\sum_{i=1}^{N}  L(\theta^{(t)}, x^{(i)})\right)}_\text{loss over all dataset examples}$$

Up until now we have worked with synthetic datasets whose size N was small. However typical deep learning datasets can have millions of examples, for which the loss is expensive to compute. It is therefore more computationally efficient to compute gradient over a smaller number of examples.

### Mini-batch gradient descent
We compute the gradient using disjoint subsets of size B from the dataset, which we call mini-batch: $x^{(1)}$, $x^{(2)}$, ..., $x^{(B)}$
$$\theta^{(t+1)} = \theta^{(t)} - α∇_\theta\underbrace{\left(\frac{1}{B}\sum_{i=1}^{B}  L(\theta^{(t)}, x^{(i)})\right)}_\text{loss over a batch of examples}$$


We iterate over the dataset mini-batches until we exhaust the dataset. One complete cycle that has seen the entire dataset is called an epoch.




## `DataLoader`
PyTorch offers an utility called [`DataLoader`](https://pytorch.org/docs/stable/data.html?highlight=dataset#torch.utils.data.DataLoader), which helps with loading mini-batches of data. For instance, if we want to load mini-batches of 100 images from the CIFAR 10 dataset, the dataloader will iterate over the whole dataset, retrieving a `Tensor` of size `100x3x32x32` at a time.

|![Minibatch](https://i.ibb.co/GTh9FY3/minibatch.png)|
|:--:|
| Mini-batch from [CIFAR 10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) |

`DataLoader` offers support for:
 - shuffling examples
 - building the mini-batch automatically, by retrieving individual examples and combining them into a single `Tensor`
 - loading the data on multiple processes

Iterating through a `DataLoader` looks like this (don't run the cell, it's just an example):


In [ ]:
# instantiate the DataLoader
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=100,
    num_workers=4,
    shuffle=True
)

for batch, labels in train_dataloader:
    # batch should be a Tensor of size 100x3x32x32
    # ...

However we need to implement the logic of retrieving an example as a `Tensor` ourselves. To do this, we implement a `Dataset`.

## [`Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) (TODO)

Machine Learning datasets are usually stored as *raw* data, which is not suitable as an input to a neural network. These *raw datasets* may be organized in many ways, such as:
 - folder with one example per file (`.jpg`, `.txt`, `.json` etc.)
 - folders where each example has its own subfolder
 - a single `.json` or `.csv` file containing all examples

Is is the practitioner's job to implement the logic of converting these *raw datasets* into a numeric format to be processed by a neural network.

To implement this logic of loading individual examples in PyTorch, we subclass the [`Dataset`](https://pytorch.org/docs/stable/data.html?highlight=dataset#torch.utils.data.Dataset) class and implement the following methods:
 - `__init__(...)`:
  - usually stores the dataset examples (if they fit into memory) or the file paths to the examples (to be later read by `__getitem()__`)
  - stores the dataset labels as well as other metadata (dictionary from label names to numbers, maximum length of an example etc.)
  - performs data preprocessing if needed
 - `__getitem(index)__`:
  - retrieves the example at position `index` in the dataset
  - transforms the example into `Tensor` format
  - returns the example and its label as `Tensor` format
 - `__len__()`:
  - returns the dataset size



We're going to implement a custom ``Dataset`` for storing some samples from the [Cats vs. Dogs](https://www.kaggle.com/c/dogs-vs-cats/data) dataset available on Kaggle.

In [ ]:
# Download the samples
!wget --no-check-certificate \
https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
-O /tmp/cats_and_dogs_filtered.zip

# Extract the data
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# set up train and validation dirs
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-03-12 18:14:36--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.234.207, 108.177.112.207, 74.125.124.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.234.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  81.8MB/s    in 0.8s    

2024-03-12 18:14:37 (81.8 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



Implement the ``CatsAndDogsDataset`` class by subclasssing the `Dataset` class:
  - the `__init__` function:
    - receives the path to the images folder `data_dir`
    - stores the images' paths in a list (to be later used by the `__getitem__` method)
    - stores the 'cat' and 'dog' labels as numbers (0 and 1) in a list
  - the `__getitem__` method:
    - retrieves the image path at the selected `index`
    - opens the image and converts it to a `Tensor`
    - retrieves the image's corresponding label
    - returns the image and its label in `Tensor` format
  - the `__len__` method:
    - returns the size of the dataset (number of files in the folder)

In [ ]:
dir_path = 'MyDocument/daniel/img_dataset'
cat: 'MyDocument/daniel/img_dataset/cats'

(dir_path, 'cats')

('MyDocument/daniel/img_dataset', 'cats')

In [ ]:
class CatsAndDogsDataset(Dataset):
  def __init__(self, data_dir: str, transform=None, cat_undersample_frac=1.0):
    # dir structure:
    # /
    #  /cats/
    #        cat.0.jpg
    #        cat.1.jpg
    #        ...
    #  /dogs/
    #        dog.0.jpg
    #        dog.1.jpg
    #        ...
    #
    self.data_dir = data_dir

    # Subdirectory containing cat pictures
    self.cats_dir = os.path.join(self.data_dir, 'cats')

    # Directory with our training dog pictures
    self.dogs_dir = os.path.join(self.data_dir, 'dogs')

    self.cat_fnames = [os.path.join(self.cats_dir, fname) \
                        for fname in os.listdir(self.cats_dir)]
    self.dog_fnames = [os.path.join(self.dogs_dir, fname) \
                        for fname in os.listdir(self.dogs_dir)]

    # subsample cat photos
    self.cat_fnames = self.cat_fnames[:int(len(self.cat_fnames) * cat_undersample_frac)]
    self.fnames = self.cat_fnames + self.dog_fnames

    self.labels = len(self.cat_fnames) * [0] + len(self.dog_fnames) * [1]

    # Every image goes through two transformations:
    # - resize all images to 375x500
    # - convert them from PIL format to Tensor
    self.image_transforms = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ])

  def __getitem__(self, index) -> (torch.Tensor, torch.Tensor):
    """
    Reads the image at position index in the self.fnames list.
    Resizes the image and converts it to a Tensor.

    Function returns a tuple of Tensors (img, label), where img
    is the image and label is the image's numerical label (0 or 1)
    """
    # TODO: select filename at position index in self.fnames
    # Solution:
    # fname = ...

    # TODO: Read image called fname using Image.open().
    # This results in a PIL image object. PIL comes from
    # Python Image Library
    # https://en.wikipedia.org/wiki/Python_Imaging_Library
    # Solution:
    # img_obj =

    # TODO: Apply self.image_transforms() on the PIL object to
    # obtain a Tensor
    # Solution:
    # img_tensor = ...

    # Tensor has shape (3, height, width), where each of
    # the first 3 slices represent the red, green and blue
    # channel values
    assert img_tensor.size() == torch.Size([3, 32, 32]), \
        "wrong image shape %s" % img_tensor.size()

    # retrieve the image's label and store it into a Tensor
    label = self.labels[index]
    label_tensor = torch.tensor([label])

    return img_tensor, label_tensor

  def __len__(self):
    """
    Return the size of this dataset. This is given by the number
    of elements in the self.fnames list.
    """
    return len(self.fnames)

Let's instantiate the training and validation datasets and print their sizes.

In [ ]:
train_dataset = CatsAndDogsDataset(data_dir=train_dir)
validation_dataset = CatsAndDogsDataset(data_dir=validation_dir)
print("Train dataset size: ", len(train_dataset))
print("Validation dataset size: ", len(validation_dataset))

Train dataset size:  2000
Validation dataset size:  1000


**We** now plot a random image from the training dataset.

In [ ]:
%matplotlib inline
random_index = random.randrange(2000)

# train_dataset[index] calls train_dataset.__getitem__(index)
random_img, label = train_dataset[random_index]

# convert one-element Tensor to scalar
label = label.item()
print("Image label: %d (%s)" % (label, 'cat' if label == 0 else 'dog'))

# check shape of Tensor holding the image
# 3 x height x width
print("Image shape: ", random_img.shape)

# permute image dimensions to (height x width x 3)
# this order is required by the plotting imshow() function
# https://matplotlib.org/3.1.3/api/_as_gen/matplotlib.pyplot.imshow.html
random_img = random_img.permute(1, 2, 0)

# plot image
plt.imshow(random_img)
plt.show()

We now plot the red, green and blue channels of the image.

In [ ]:
# plot red channel
plt.imshow(random_img[:,:,0], cmap='gray')
plt.show()

# plot green channel
plt.imshow(random_img[:,:,1], cmap='gray')
plt.show()

# plot blue channel
plt.imshow(random_img[:,:,2], cmap='gray')

## Loading the mini-batches
Let's now instantiate one `DataLoader` each for the training and validation datasets.

In [ ]:
BATCH_SIZE=32

# instantiate the DataLoaders
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
validation_dataloader = DataLoader(
    dataset=validation_dataset,
    batch_size=BATCH_SIZE
)

Let's iterate through the training dataset and inspect the batches:

In [ ]:
%matplotlib inline
print_full_batch = True
for batch in train_dataloader:
    batch_img, batch_labels = batch
    print("Batch of images size: ", batch_img.size())
    print("Batch of labels size: ", batch_labels.size())

    if print_full_batch:
        fig, axes = plt.subplots(
            nrows=BATCH_SIZE//4,
            ncols=4,
            figsize=(10,10)
        )
        for idx, image in enumerate(batch_img):
            row = idx // 4
            col = idx % 4
            axes[row, col].axis("off")
            axes[row, col].imshow(image.permute(1, 2, 0), cmap="gray", aspect="auto")
        plt.subplots_adjust(wspace=.05, hspace=.05)
        plt.show()
    else:
        # print random image from batch
        random_index = random.randrange(32)
        random_img, label = batch_img[random_index], batch_labels[random_index]
        label = label.item()
        print("Image label: %d (%s)" % (label, 'cat' if label == 0 else 'dog'))

        # permute image dimensions to (height x width x 3)
        # this order is required by the plotting imshow() function
        # https://matplotlib.org/3.1.3/api/_as_gen/matplotlib.pyplot.imshow.html
        random_img = random_img.permute(1, 2, 0)

        # plot image
        plt.imshow(random_img)
        plt.show()

    break

## Create Model (TODO)

Let's now create an MLP to solve this classification task. The network has 2 hidden layers.

In [ ]:
class CatsAndDogsClassifier(nn.Module):
    def __init__(self,
                 input_size: int,
                 hidden_size_1: int,
                 hidden_size_2: int,
                 activation_fn: Callable):
        super().__init__()
        self.input_size = input_size
        self.hidden_size_1 = hidden_size_1
        self.hidden_size_2 = hidden_size_2

        #TODO: instantiate layers
        self.layer_1 = ...
        self.layer_2 = ...
        self.output_layer = ...

        self.activation_fn = activation_fn

    def forward(self, x):
        # TODO: layer1 + activation
        # h1 = ...

        # TODO: layer2 + activation
        # h2 = ...

        # TODO: output layer
        # out = ...

        return out

Let's instantiate the model, print its layers and count the total number of parameters.

In [ ]:
model = CatsAndDogsClassifier(
    input_size=3*32*32,
    hidden_size_1=768,
    hidden_size_2=512,
    activation_fn=nn.ReLU()
)
num_params = 0
print("Model's parameters: ")
for n, p in model.named_parameters():
    print('\t', n, ': ', p.size())
    num_params += p.numel()
print("Number of model parameters: ", num_params)


Model's parameters: 
	 layer_1.weight :  torch.Size([768, 3072])
	 layer_1.bias :  torch.Size([768])
	 layer_2.weight :  torch.Size([512, 768])
	 layer_2.bias :  torch.Size([512])
	 output_layer.weight :  torch.Size([2, 512])
	 output_layer.bias :  torch.Size([2])
Number of model parameters:  2754818


## Training and validation loops (TODO)

Let's set up training and validation loops. Complete the `train_epoch()` and `validation_epoch()` functions below.

In [ ]:
def train_epoch(model, train_dataloader, loss_crt, optimizer, device):
    """
    model: Model object
    train_dataloader: DataLoader over the training dataset
    loss_crt: loss function object
    optimizer: Optimizer object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch training loss, which is an average over the individual batch
       losses
     - the predictions made by the model
     - the labels
    """
    model.train()
    epoch_loss = 0.0
    num_batches = len(train_dataloader)
    predictions = []
    labels = []
    for idx, batch in tqdm(enumerate(train_dataloader)):
        # batch_size x 3 x 32 x 32, batch_size x 1
        batch_img, batch_labels = batch
        current_batch_size = batch_img.size(0)

        # move data to GPU
        batch_img = batch_img.to(device)
        batch_labels = batch_labels.to(device).squeeze()

        # resize images: batch_size x 3 x 32 x 32 => batch_size x (3 x 32 x 32)
        batch_img = batch_img.view(current_batch_size, -1)

        # batch_size x 2
        # TODO: feedforward through the model
        # output = ...

        # batch_size
        # TODO: model's predictions for this batch
        # Hint: use torch.argmax to get the class the model assigned a higher confidence
        # batch_predictions = ...

        # save batch predictions and labels
        predictions += batch_predictions.tolist()
        labels += batch_labels.tolist()

        # TODO: compute loss for current batch
        # loss = ...

        loss_scalar = loss.item()

        # TODO: compute gradients
        # ...

        # TODO: update parameters
        # ...

        # TODO: reset gradients
        # ...

        epoch_loss += loss_scalar

    epoch_loss = epoch_loss/num_batches

    return epoch_loss, predictions, labels

def eval_epoch(model, val_dataloader, loss_crt, device):
    """
    model: Model object
    val_dataloader: DataLoader over the validation dataset
    loss_crt: loss function object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch validation loss, which is an average over the individual batch
       losses
     - the predictions made by the model
     - the labels
    """
    model.eval()
    epoch_loss = 0.0
    num_batches = len(val_dataloader)
    predictions = []
    labels = []
    with torch.no_grad():
        for idx, batch in tqdm(enumerate(val_dataloader)):
            # batch_size x 3 x 32 x 32, batch_size x 1
            batch_img, batch_labels = batch
            current_batch_size = batch_img.size(0)

            # move data to GPU
            batch_img = batch_img.to(device)
            batch_labels = batch_labels.to(device).squeeze()

            # resize images: batch_size x 3 x 32 x 32 => batch_size x (3 x 32 x 32)
            batch_img = batch_img.view(current_batch_size, -1)

            # TODO: feedforward
            # batch_size x 2
            # output = ...

            # TODO: model's predictions for this batch
            # batch_size
            # batch_predictions = ...

            predictions += batch_predictions.tolist()
            labels += batch_labels.tolist()

            # TODO: compute loss for this batch
            # loss = ...

            loss_scalar = loss.item()
            epoch_loss += loss_scalar

    epoch_loss = epoch_loss/num_batches

    return epoch_loss, predictions, labels

## Compute metrics and confusion matrix (TODO)

The accuracy can be problematic when evaluating  a classification problem with ***imbalanced*** classes or when there are more than 2 classes.

The confusion matrix allows us to easily inspect the model's mistakes by providing the number of predictions broken down by class.

|![Minibatch](https://miro.medium.com/max/1400/1*n9TxgFX0_7pSLwEEYgFIHQ.png)|
|:--:|
| Confusion matrix for a cats vs dogs classifier ([source](https://medium.com/@apiltamang/case-study-a-world-class-image-classifier-for-dogs-and-cats-err-anything-9cf39ee4690e)) |


Implement the `compute_accuracy()` and `compute_confusion_table()` below.

In [ ]:
def compute_accuracy(predictions: List[int], labels:List[int]) -> float:
    """
    Compute accuracy given the predictions of a binary classifier and the
    ground truth label.
    predictions: list of model predictions (0 or 1)
    labels: list of ground truth labels (0 or 1)
    """
    # TODO:

    return epoch_accuracy

def compute_confusion_matrix(predictions: List[int], labels:List[int]) -> Tuple[int]:
    """
    Compute the confusion matrix.
    Arguments:
        predictions: list of model predictions (0 for cats or 1 for dogs)
        labels: list of ground truth labels (0 or 1)
    """
    correct_cats = 0
    correct_dogs = 0
    wrong_cats = 0
    wrong_dogs = 0
    # TODO:

    return (correct_cats, wrong_cats, correct_dogs, wrong_dogs)

## Training the model

In [ ]:
hyperparams = {
    'optimizer': 'adam',
    'lr': 0.001,
    'momentum': 0.0,
    'num_epochs': 30,
    'hidden_size_1': 768,
    'hidden_size_2': 512,
    'activation_fn': nn.ReLU(),
}

# set up loss function
loss_criterion = nn.CrossEntropyLoss()

# instantiate model and move to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CatsAndDogsClassifier(
    input_size=3*32*32,
    hidden_size_1=hyperparams['hidden_size_1'],
    hidden_size_2=hyperparams['hidden_size_2'],
    activation_fn=hyperparams['activation_fn']
)
model.to(device)

# instantiate optimize
if hyperparams['optimizer'] == 'adam':
    optimizer = torch.optim.Adam(
        params=model.parameters(),
        lr=hyperparams['lr']
    )
elif hyperparams['optimizer'] == 'sgd':
    optimizer = torch.optim.SGD(
        params=model.parameters(),
        lr=hyperparams['lr'],
        momentum=hyperparams['momentum']
    )

train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []
best_val_acc = 0.0

for epoch_idx in range(hyperparams['num_epochs']):
    train_epoch_loss, train_predictions, train_labels = train_epoch(
        model,
        train_dataloader,
        loss_criterion,
        optimizer,
        device
    )
    val_epoch_loss, val_predictions, val_labels = eval_epoch(
        model,
        validation_dataloader,
        loss_criterion,
        device
    )
    train_acc = compute_accuracy(train_predictions, train_labels)
    val_acc = compute_accuracy(val_predictions, val_labels)
    train_losses.append(train_epoch_loss)
    val_losses.append(val_epoch_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print("epoch %d, train loss=%f, train acc=%f, val loss=%f, val acc=%f" % (
        epoch_idx,
        train_epoch_loss,
        train_acc,
        val_epoch_loss,
        val_acc
    ))


## Model evaluation



### Plotting the learning curves and accuracy curves

In [ ]:
%matplotlib inline
NUM_EPOCHS=hyperparams['num_epochs']
plt.plot(range(0,len(train_losses)), train_losses, 'g', label='Training loss')
plt.plot(range(0,len(train_losses)), val_losses, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(range(0,len(train_accuracies)), train_accuracies, 'g', label='Training accuracy')
plt.plot(range(0,len(train_accuracies)), val_accuracies, 'b', label='Validation accuracy')
plt.title('Training and Validation accuracies')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Plotting the confusion matrix

In [ ]:
correct_cats, wrong_cats, correct_dogs, wrong_dogs = compute_confusion_matrix(
    val_predictions,
    val_labels
)
cf_matrix = [
    [correct_cats, wrong_cats],
    [correct_dogs, wrong_dogs]
]
import seaborn as sns

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='d')

ax.set_title('Confusion Matrix\n\n');
ax.set_xlabel('Actual Values')
ax.set_ylabel('Predicted Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['cat','dog'])
ax.yaxis.set_ticklabels(['cat','dog'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
cf_matrix

### Gradient norms
We will now inspect the evolution of the gradient norms and of the ratio m/v for the Adam optimizer terms throughout training.

TODO: modify the `train_epoch` function implemented in the previous task such that we also save the gradient norms and the ratios between the first and second order momentum averages of the Adam optimizer.

In [ ]:
def train_epoch(model, train_dataloader, loss_crt, optimizer, device):
    """
    model: Model object
    train_dataloader: DataLoader over the training dataset
    loss_crt: loss function object
    optimizer: Optimizer object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch training loss, which is an average over the individual batch
       losses
     - the predictions made by the model
     - the labels
    """
    model.train()
    epoch_loss = 0.0
    num_batches = len(train_dataloader)
    predictions = []
    labels = []
    grad_norms = []
    m_v_ratios = []
    eps = 0.000001

    for idx, batch in tqdm(enumerate(train_dataloader)):
        # batch_size x 3 x 32 x 32, batch_size x 1
        batch_img, batch_labels = batch
        current_batch_size = batch_img.size(0)

        # move data to GPU
        batch_img = batch_img.to(device)
        batch_labels = batch_labels.to(device)

        # resize images: batch_size x 3 x 32 x 32 => batch_size x (3 x 32 x 32)
        batch_img = batch_img.view(current_batch_size, -1)

        # batch_size x 2
        # TODO: feedforward through the model
        # output = ...

        # batch_size
        # TODO: model's predictions for this batch
        # batch_predictions = ...

        # save batch predictions and labels
        predictions += batch_predictions.tolist()
        labels += batch_labels.tolist()

        # TODO: compute loss for current batch
        # loss = ...

        loss_scalar = loss.item()

        # TODO: compute gradients
        # ...

        grad_norms_batch = []
        m_v_ratios_batch = []

        # TODO: compute the norm of the gradients
        # Hint: iterate through model.parameters() and use the .grad property saved by autograd
        grad_norm = ...
        grad_norms.append(grad_norm)

        # TODO: update parameters
        # ...

        # TODO: compute the m / v ratio for the weights of the first fully-connected layer
        # Hint: check out what optimizer.state_dict() stores after calling optimizer.step()
        m_v_ratios_batch = []
        for idx_module in range(len(list(model.parameters()))):
            exp_avg = ...
            exp_avg_sq = ...
            ## we will add a small eps to the denominator to avoid zero division

            # TODO compute the element-wise ratio of m / (v+eps)
            # compute mean() on the resulting tensor such that ratio contains a single value
            ratio = ...
            m_v_ratios_batch.append(ratio)

        m_v_ratios.append(torch.mean(torch.stack(m_v_ratios_batch), dim=0))

        # TODO: reset gradients
        # ...
        epoch_loss += loss_scalar

    epoch_loss = epoch_loss/num_batches

    # TODO compute grad norm mean
    grad_norms_mean = torch.mean(torch.stack(grad_norms)).cpu().numpy()
    m_v_ratios_mean = torch.mean(torch.stack(m_v_ratios)).cpu().numpy()

    return epoch_loss, predictions, labels, grad_norms_mean, m_v_ratios_mean

We train the network again for a few epochs

In [ ]:
hyperparams = {
    'optimizer': 'adam',
    'lr': 0.001,
    'momentum': 0.0,
    'num_epochs': 10,
    'hidden_size_1': 768,
    'hidden_size_2': 512,
    'activation_fn': nn.ReLU(),
}

# set up loss function
loss_criterion = nn.CrossEntropyLoss()

# instantiate model and move to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CatsAndDogsClassifier(
    input_size=3*32*32,
    hidden_size_1=hyperparams['hidden_size_1'],
    hidden_size_2=hyperparams['hidden_size_2'],
    activation_fn=hyperparams['activation_fn']
)
model.to(device)

# instantiate optimize
if hyperparams['optimizer'] == 'adam':
    optimizer = torch.optim.Adam(
        params=model.parameters(),
        lr=hyperparams['lr']
    )
elif hyperparams['optimizer'] == 'sgd':
    optimizer = torch.optim.SGD(
        params=model.parameters(),
        lr=hyperparams['lr'],
        momentum=hyperparams['momentum']
    )

train_losses, val_losses, grad_norms, m_v_ratios = [], [], [], []
train_accuracies, val_accuracies = [], []
best_val_acc = 0.0

for epoch_idx in range(hyperparams['num_epochs']):
    train_epoch_loss, train_predictions, train_labels, grad_norms_epoch, m_v_ratios_epoch = train_epoch(
        model,
        train_dataloader,
        loss_criterion,
        optimizer,
        device
    )
    val_epoch_loss, val_predictions, val_labels = eval_epoch(
        model,
        validation_dataloader,
        loss_criterion,
        device
    )
    train_acc = compute_accuracy(train_predictions, train_labels)
    val_acc = compute_accuracy(val_predictions, val_labels)
    val_losses.append(val_epoch_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    grad_norms.append(grad_norms_epoch)
    m_v_ratios.append(m_v_ratios_epoch)

    print("epoch %d, train loss=%f, train acc=%f, val loss=%f, val acc=%f" % (
        epoch_idx,
        train_epoch_loss,
        train_acc,
        val_epoch_loss,
        val_acc
    ))

We plot the gradient norms and observe their evolution throughout training. We observe that the gradient norms are initially high, then decreasing throughout training.

In [ ]:
plt.plot(range(0, len(grad_norms)), grad_norms, 'g', label='Gradient Norms')
plt.title('Gradient norms')
plt.xlabel('Epochs')
plt.legend()
plt.show()

We plot the m/(v+eps) ratios used by the Adam optimizer as learning rate adapters. We observe that the ratio decreases over time, allowing the optimizer to make smaller steps when the networks is getting closer to an optimal point.

In [ ]:
plt.plot(range(0, len(m_v_ratios)), m_v_ratios, 'b', label='m / (v + eps)')
plt.title('m/(v+eps)')
plt.xlabel('Epochs')
plt.legend()
plt.show()

## Unbalanced datasets

So far we used a balanced dataset, with equal number of examples for our two classes: `cat` and `dog`. If our dataset is highly imbalanced, we bias our prediction towards the examples that the models has seen more often.

In this section, we will use an imbalanced train dataset and weight the classes in the optimizer, using the inverse frequencies of the two classes.

In [ ]:
# We undersample the cat labeled examples and only use half of them
cat_undersample_frac=0.5
train_dataset = CatsAndDogsDataset(data_dir=train_dir, cat_undersample_frac=cat_undersample_frac)
validation_dataset = CatsAndDogsDataset(data_dir=validation_dir)
print("Train dataset size: ", len(train_dataset))
print("Validation dataset size: ", len(validation_dataset))

In [ ]:
BATCH_SIZE=32

# instantiate the DataLoaders
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
validation_dataloader = DataLoader(
    dataset=validation_dataset,
    batch_size=BATCH_SIZE
)

First, let's train the network using the default parameters of the optimzier.

In [ ]:
hyperparams = {
    'optimizer': 'adam',
    'lr': 0.001,
    'momentum': 0.0,
    'num_epochs': 20,
    'hidden_size_1': 768,
    'hidden_size_2': 512,
    'activation_fn': nn.ReLU(),
}

# set up loss function
loss_criterion = nn.CrossEntropyLoss()

# instantiate model and move to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CatsAndDogsClassifier(
    input_size=3*32*32,
    hidden_size_1=hyperparams['hidden_size_1'],
    hidden_size_2=hyperparams['hidden_size_2'],
    activation_fn=hyperparams['activation_fn']
)
model.to(device)

# instantiate optimize
if hyperparams['optimizer'] == 'adam':
    optimizer = torch.optim.Adam(
        params=model.parameters(),
        lr=hyperparams['lr']
    )
elif hyperparams['optimizer'] == 'sgd':
    optimizer = torch.optim.SGD(
        params=model.parameters(),
        lr=hyperparams['lr'],
        momentum=hyperparams['momentum']
    )

train_losses, val_losses = [], []
train_accuracies, val_accuracies_unbalanced = [], []
best_val_acc = 0.0

for epoch_idx in range(hyperparams['num_epochs']):
    train_epoch_loss, train_predictions, train_labels, _, _ = train_epoch(
        model,
        train_dataloader,
        loss_criterion,
        optimizer,
        device
    )
    val_epoch_loss, val_predictions, val_labels = eval_epoch(
        model,
        validation_dataloader,
        loss_criterion,
        device
    )
    train_acc = compute_accuracy(train_predictions, train_labels)
    val_acc = compute_accuracy(val_predictions, val_labels)
    val_losses.append(val_epoch_loss)
    train_accuracies.append(train_acc)
    val_accuracies_unbalanced.append(val_acc)

    print("epoch %d, train loss=%f, train acc=%f, val loss=%f, val acc=%f" % (
        epoch_idx,
        train_epoch_loss,
        train_acc,
        val_epoch_loss,
        val_acc
    ))

We observe that the validation loss does not decrease too much in the above example. Let's pass the class weights as inverse frequencies to the optimizer.

TODO: add weights to each class to the CrossEntropy loss function. We can use `1/pC`, where `pC` is the occurrence probability of class `p`.

In [ ]:
hyperparams = {
    'optimizer': 'adam',
    'lr': 0.001,
    'momentum': 0.0,
    'num_epochs': 20,
    'hidden_size_1': 768,
    'hidden_size_2': 512,
    'activation_fn': nn.ReLU(),
}

# instantiate model and move to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# set up loss function

# TODO add class weights to the CorssEntropy loss
loss_criterion = ...

model = CatsAndDogsClassifier(
    input_size=3*32*32,
    hidden_size_1=hyperparams['hidden_size_1'],
    hidden_size_2=hyperparams['hidden_size_2'],
    activation_fn=hyperparams['activation_fn']
)
model.to(device)

# instantiate optimize
if hyperparams['optimizer'] == 'adam':
    optimizer = torch.optim.Adam(
        params=model.parameters(),
        lr=hyperparams['lr']
    )
elif hyperparams['optimizer'] == 'sgd':
    optimizer = torch.optim.SGD(
        params=model.parameters(),
        lr=hyperparams['lr'],
        momentum=hyperparams['momentum']
    )

train_losses, val_losses = [], []
train_accuracies, val_accuracies_weighted = [], []
best_val_acc = 0.0

for epoch_idx in range(hyperparams['num_epochs']):
    train_epoch_loss, train_predictions, train_labels, _, _ = train_epoch(
        model,
        train_dataloader,
        loss_criterion,
        optimizer,
        device
    )
    val_epoch_loss, val_predictions, val_labels = eval_epoch(
        model,
        validation_dataloader,
        loss_criterion,
        device
    )
    train_acc = compute_accuracy(train_predictions, train_labels)
    val_acc = compute_accuracy(val_predictions, val_labels)
    val_losses.append(val_epoch_loss)
    train_accuracies.append(train_acc)
    val_accuracies_weighted.append(val_acc)

    print("epoch %d, train loss=%f, train acc=%f, val loss=%f, val acc=%f" % (
        epoch_idx,
        train_epoch_loss,
        train_acc,
        val_epoch_loss,
        val_acc
    ))

Finally, let's compare the two validation loss curves.

In [ ]:
plt.plot(range(0,len(val_accuracies_unbalanced)), val_accuracies_unbalanced, 'g', label='unbalanced')
plt.plot(range(0,len(val_accuracies_weighted)), val_accuracies_weighted, 'b', label='weighted')
plt.title('Validation accuracies')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()